In [1]:
from collections import OrderedDict
import datetime as dt
from pathlib import Path

import pandas as pd
import numpy as np
from highcharts import Highchart



In [2]:
# A DataFrame of *transactions* is one that contains at least these columns:
KEY_COLUMNS = {'date', 'credit', 'debit', 'balance'}

def find_key_columns(raw_transactions):
    """
    Given a DataFrame, try to find in it the columns that correspond those in ``KEY_COLUMNS``.
    Build a dictionary of this correspondance of the form
    key column name -> column name in given DataFrame.
    Return the resulting dictionary, which might be incomplete.
    """
    f = raw_transactions.copy()
    col_dict = {}
    for c in f.columns:
        for key in KEY_COLUMNS:
            if key in c.lower():
                col_dict[key] = c
    return col_dict

def read_transactions(path, date_format='%d-%m-%Y'):
    """
    Given a path to a CSV file of bank transactions, read the file, and 
    convert it to a DataFrame of transactions, that is, 
    a DataFrame with at least the columns in ``KEY_COLUMNS``.
    Parse the dates in CSV file according to the format ``date_format``, and 
    preserve any data in extra columns. 
    Return the resulting DataFrame.
    """
    f = pd.read_csv(path)
    col_dict = find_key_columns(f)
    if set(col_dict.keys()) != KEY_COLUMNS:
        raise ValueError('Could not find columns resembling {!s} in file'.format(
          KEY_COLUMNS))
    
    # Reformat column names
    rename1 = {val: key for key, val in col_dict.items()}
    rename2 = {c: c.strip().lower().replace(' ', '_') for c in f.columns}
    f = f.rename(columns=rename1).rename(columns=rename2)
    
    # Parse dates
    f['date'] = pd.to_datetime(f['date'], format=date_format)
    
    return f

def make_sample_transactions(start_date, end_date, freq='3D'):
    """
    Generate a DataFrame of transactions from the start date to the end date
    (date strings) with credits and debits at the given frequency (Pandas frequency string).
    """
    r = pd.date_range(start_date, end_date, freq='3D')
    f = pd.DataFrame(r, columns=['date'])
    n = len(r)
    f['credit'] = np.random.randint(0, 100, n)
    f['debit'] = np.random.randint(0, 100, n)
    f['balance'] = (f['credit'] - f['debit']).cumsum()
    return f

def summarize(transactions, freq='MS'):
    """
    Given a DataFrame of transactions summarize it at the given frequency (Pandas frequency string),
    summing credits, summing debits, and taking the last balance for each period.
    Return the resulting DataFrame.
    """
    cols = ['date', 'credit', 'debit', 'balance'] 
    f = transactions[cols].copy()
    if freq is None:
        g = {}
        g['date'] = f['date'].min()
        g['credit'] = f['credit'].sum()
        g['debit'] = f['debit'].sum()
        g['balance'] = f['balance'].iat[-1]
        g = pd.DataFrame(g, index=[0])
    else:
        g = f.set_index('date').resample(freq).agg({
          'credit': 'sum',
          'debit': 'sum',
          'balance': 'last', 
          }).fillna(0).reset_index()
    
    return g[cols].copy()

def plot(summary, currency='NZD', width=700, height=None):
    """
    Given a transaction summary of the form output by :func:`summarize`, plot it using Python HighCharts.
    Include the given currency units (string; e.g. 'NZD') in the plot labels.
    """
    f = summary.copy()
    chart = Highchart()

    # HighCharts kludge: use categorical x-axis to display dates properly
    dates = f['date'].map(lambda x:x.strftime('%Y-%m-%d')).unique()
    dates = sorted(dates.tolist())

    if currency:
        y_text = 'Money ({!s})'.format(currency)
    else:
        y_text = 'Money'
    
    options = {
        'lang': {
            'thousandsSep': ','
        },
        'chart' : {},
        'title': {
            'text': 'Account Summary'
        },
        'xAxis': {
            'type': 'category',
            'categories': dates,
        },
        'yAxis': {
            'title': {
                'text': y_text,
            }
        },
        'tooltip': {
            'headerFormat': '<b>{point.key}</b> ' +
              '(period start)<table>',
            'pointFormat': '''
              <tr>
              <td style="padding-right:1em">{series.name}</td>
              <td style="text-align:right">{point.y:,.0f} ''' + currency +\
              '''
              </td>
              </tr>
              ''',    
            'useHTML': True,
            'shared': True,
        },
        'plotOptions': {
            'column': {
                'pointPadding': 0,
                'borderWidth': 1,
                'borderColor': '#333333',
            }
        },
        'credits': {
                'enabled': False,
            },
    }

    if width is not None:
        options['chart']['width'] = width

    if height is not None:
        options['chart']['height'] = height

    chart.set_dict_options(options)
    for (col, opts) in [
      ('credit', {'series_type': 'column', 'color': '#8da0cb'}),
      ('debit', {'series_type': 'column', 'color': '#fc8d62'}),
      ('balance', {'series_type': 'line', 'color': '#555'}),
      ]:
        chart.add_data_set(f[col].values.tolist(), name=col, **opts)

    return chart

In [3]:
# Use mock data

transactions = make_sample_transactions('2015-01-01', '2017-01-01')
transactions.tail()

,date,credit,debit,balance
239,2016-12-18,66,74,-786
240,2016-12-21,21,90,-855
241,2016-12-24,2,33,-886
242,2016-12-27,4,59,-941
243,2016-12-30,49,81,-973


In [ ]:
# Or load real data

DATA_DIR = Path('~')/'tumeke_cycle_space'/'finances'
paths = [
    'transactions_20140701--20150630.csv',
    'transactions_20150701--20160630.csv',
    'transactions_20160701--20161228.csv',
    'transactions_20161201--20170502.csv',    
    'transactions_20170501--20170913.csv',
]
paths = [DATA_DIR/p for p in paths]
    
frames = [read_transactions(p) for p in paths]
transactions = pd.concat(frames).drop_duplicates()
transactions.tail().T


In [4]:
# Summarize and plot

summary = summarize(transactions, freq='MS')
print(summary)
chart = plot(summary)
chart

         date  credit  debit  balance
0  2015-01-01     569    644      -75
1  2015-02-01     386    454     -143
2  2015-03-01     470    411      -84
3  2015-04-01     611    669     -142
4  2015-05-01     463    508     -187
5  2015-06-01     585    556     -158
6  2015-07-01     477    724     -405
7  2015-08-01     346    482     -541
8  2015-09-01     406    437     -572
9  2015-10-01     502    564     -634
10 2015-11-01     679    562     -517
11 2015-12-01     579    555     -493
12 2016-01-01     343    465     -615
13 2016-02-01     682    501     -434
14 2016-03-01     325    591     -700
15 2016-04-01     532    617     -785
16 2016-05-01     429    484     -840
17 2016-06-01     517    592     -915
18 2016-07-01     490    443     -868
19 2016-08-01     594    458     -732
20 2016-09-01     473    448     -707
21 2016-10-01     465    432     -674
22 2016-11-01     371    483     -786
23 2016-12-01     468    655     -973


In [ ]:
# Save plot to HTML file

path = OUT_DIR/'account_summary_20140701--20170913'

if not path.parent.exists():
    path.parent.mkdir(parents=True)

chart.save_file(str(path))
